# Estatísticas Descritivas

Uma das principais ferramentas na investigação de um conjunto de dados são as **estatísticas descritivas**. Estatísticas descritivas são **funções** (no sentido matemático) que resumem um vetor de dados (por exemplo, uma coluna do Pandas) em **um número**. O exemplo clássico de estatística descrita é a **média**: dado um conjunto de observações (alturas de indivíduos), podemos calcular a **altura média** desses indivíduos.



Além de serem úteis na exploração de um conjunto de dados, as estatísticas descritivas nos permitem fazer **inferências** sobre uma população a partir de uma amostra dessa população: 

> Um cientista político deseja descobrir como a população de um país se sente com relação à diferentes candidatos em uma eleição, pedindo que cada pessoa de uma nota de 0 à 10 para cada candidato. Obviamente, este cientista não tem como entrevistar todas as pessoas de um país, ele precisará conduzir sua entrevista para uma amostra da população. A **inferência estatística** nos permite estimar as estatísticas descritivas de uma população a partir dessas mesmas estatísticas em uma amostra. Desta forma, o cientista poderá **estimar**, com grau de confiabilidade conhecido, as opções da população como um todo a partir de uma amostra aleatória.

A **inferência estatística** nos permite estimar dados de uma população através de observações conduzidas sobre uma amostra. O caminho reverso, a **amostragem**, nos permite criar observações a partir de uma população conhecida. Este tipo de análise é usado predominantemente de duas: 

1. **DOEs** (*design of experiments*): utilizamos estimativas dos parâmetros de uma população para criar uma estratégia de amostragem em um experimento;
    * No exemplo acima, o cientista político precisa calcular o grau de confiabilidade para diferentes tamanhos de amostra.
1. **Simulação de Monte Carlo**: utilizamos estimativas dos parâmetros de uma população para simular um processo.
    * Podemos utilizar parâmetros mensurados em um cruzamento para simular como diferentes volumetrias de carro podem impactar o trânsito em uma avênida.

![image.png](images/inference.png)

## Geyser dataset

Vamos analisar como diferentes estatísticas descritivas podem ser utilizadas e quando devemos evitar uma ou outra medida. Para essa aula utilizaremos o conjunto de dados `Geyser` que contém três variáveis relacionadas à erupções do geiser *Old Faithful* no parque de Yellowstone nos EUA:

1. O tempo desde a última erupção (variável continua);
1. A duração da erupção (variável continua);
1. O tipo de erupção (variável discreta).

Fonte: https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/faithful.html

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
sns.set_theme('paper')

In [ ]:
tb_geyser = sns.load_dataset('geyser')

In [ ]:
tb_geyser.head()

In [ ]:
tb_geyser.describe()

## Localização

As primeiras estatísticas descritivas de hoje serão as as **medidas de localização** (ou *medidas de altura*). Essas medidas nos permitem estimar ao redor que qual ponto uma distribuição se concentram. Focaremos nossa análise em três medidas específicas:

1. **Média** (ou *esperança*);
1. **Mediana** (o *segundo quartil*);
1. **Moda**

### Média

A **média** é a medida de localização mais conhecida (e, provavelmente, o conceito estatístico mais difundido). Ela é definida como a somatória de todas as observações divida pelo número de observações e, na estatítisca, é comunmente representada pela letra Mu:

$$\mu=\frac{\sum_{i=1}^n x_{i}}{n} $$

Vamos começar construindo uma função para calcular a média:


In [ ]:
def meu_mu(vetor):
    total = sum(vetor)
    numero_elementos = len(vetor)
    media = total/numero_elementos
    return media

In [ ]:
meu_mu(tb_geyser['waiting'])

Podemos utilizar a função `mean()` para calcular a média de qualquer iterável numérico no Python:

In [ ]:
np.mean(tb_geyser['waiting'])

ou então podemos utilizar o método `.mean()` para calcular a média de uma (ou mais) colunas de um `DataFrame`:

In [ ]:
tb_geyser['waiting'].mean()

In [ ]:
tb_geyser[['waiting','duration']].mean()

In [ ]:
tb_geyser.shape

#### Limitações da Média

Toda estatística descritiva é um extremamente reducionista: no caso acima estamos **reduzindo 272 observações a apenas um número!** Essa redução é comum à todas as estatísticas descritivas e por isso é extramemente importante conhecermos as limitações de cada uma dessas funções.

Vamos começar analisando as **duas principais limitações da média**: distribuições multi-modais e sensibilidade à outliers.

##### Multimodalidade

Muitas vezes uma variável contém observações de duas populações distintas: por exemplo, a altura de homens e mulheres. Vamos ver como isso acontece em nosso dataset:

In [ ]:
tb_geyser['duration'].mean()

In [ ]:
sns.histplot(data = tb_geyser, x = 'duration').set(title='Distribuição de Durações das Erupções')
plt.axvline(tb_geyser['duration'].mean(), color = 'red');

Como podemos ver, a média da duração das erupções está localizada em uma região com pouquissímas observações! Na verdade, nossa variável tem uma distribuição **bimodal**: aparentemente temos uma mistura de duas distribuições.

Neste caso, seria apropriado analisar a nossa variável **condicionada** à outra variável que a separe as duas distribuições:

In [ ]:
sns.histplot(data = tb_geyser, x = 'duration', hue = 'kind').set(title='Distribuição de Durações das Erupções')
mask_short = tb_geyser['kind'] == 'short'
plt.axvline(tb_geyser.loc[mask_short,'duration'].mean(), color = 'orange');
plt.axvline(tb_geyser.loc[~mask_short,'duration'].mean(), color = 'blue');

In [ ]:
tb_geyser.groupby('kind')['duration'].mean()

##### Sensibilidade à Outliers

Outliers são observações que se destacam em uma amostra por terem valores extremos. Eles podem ocorrer por razões reais (a distribuição de renda por exemplo) ou artificiais (erros na coleta de dados).

Para entendermos o impacto de observações extremas sobre a média vamos utilizar o conceito da **contribuição à média** de cada observação:

$$\mu=\frac{\sum_{i=1}^n x_{i}}{n} = \frac{x_{1}}{n}+\frac{x_{2}}{n}+\frac{x_{3}}{n}+...+\frac{x_{n}}{n}$$

Cada observação contribui seu valor dividido pelo número de observações à média, ou seja, quanto maior a observação e menor a amostra, mais *peso* os valores extremos terão sobre a média! Vamos ver esse comportamento sobre um conjunto sintético de dados:

In [ ]:
a = [1, 1, 1, 2, 
     1, 1, 1, 2,
     1, 1, 1, 2,
     1, 1, 1, 2,
     10000]
np.mean(a)

A lista acima contém 12 observações com valor *1*, 4 com valor *2* e 1 com valor *10000*. Podemos utilizar a fórmula acima para calcular o quanto cada observação contribui para a média:

In [ ]:
print(f"Observações com valor 1 contribuiram {12 * 1 /len(a)} p/ a média")
print(f"Observações com valor 2 contribuiram {4 * 2/len(a)} p/ a média")
print(f"Observações com valor 10.000 contribuiram {1 * 10000/len(a)} p/ a média")


In [ ]:
(1 * 10000/len(a))/np.mean(a)

Podemos ver que a última observação, com valor 10000 é responsável por mais que 99% do valor da média! Essa sensibilidade da média faz com que ela não seja uma boa descrição para conjuntos com outliers. Vamos verificar isso criando o conjunto `b`, com todas as observações de `a` exceto aquela com valor 10.000, e calcular a média deste novo conjunto.

In [ ]:
b = [1, 1, 1, 2, 
     1, 1, 1, 2,
     1, 1, 1, 2,
     1, 1, 1, 2]
np.mean(b)

Como podemos tratar casos como este? Um caminho seria excluir valores extremos, mas devemos evitar excluir valores sem antes ter uma compreensão mais profunda sobre um conjunto de dados. Para resolver este problema podemos utilizar outra medida de localização: **a mediana**!

### Mediana

Enquanto a média é um cálculo feito a partir dos valores das observações, a mediana é um algoritmo. Quando calculamos a mediana estamos aplicando as etapas abaixo:

1. **Ordenamos** nossas observações **da menor para maior**;
1. **Contamos** o número de observações (**vamos chamá-lo de *n***);
1. **Procuramos** em nossa lista de observações ordenadas **a observação com índice *n/2***, ou seja, o elemento no meio da lista ordenada;
1. **A mediana é o valor deste observação**.

Vamos construir uma função para encontrar a mediana de um vetor de observações:

In [ ]:
def minha_mediana(vetor):
    # ORDENAMOS
    vetor_ordenado = sorted(list(vetor))
    
    # CONTAMOS
    posicao_meio = int(len(vetor_ordenado)/2)

    # PROCURAMOS
    mediana = vetor_ordenado[posicao_meio]
    return mediana

In [ ]:
minha_mediana(a)

**Porque a mediana resolve o problema de sensibilidade à outliers?**

Podemos calcular a mediana utilizando nossa própria função:

In [ ]:
minha_mediana(tb_geyser['duration'])

ou podemos utilizar a função `median()`

In [ ]:
np.median(tb_geyser['duration'])

ou então usando o método `.median()` dos `DataFrames`

In [ ]:
tb_geyser['duration'].median()

#### Limitações da Mediana

A mediana possui duas limitações importantes:

1. **Multi-modalidade** dos dados (assim como a média);
1. Solução algoritmica.

A segunda limitação é mais teórica que prática: mesmo em dados muito extensos o cálculo da mediana é rápido e, a não ser que estejamos desenvolvendo algoritmos de machine learning, raramente ela será um problema! Vamos focar nossos esforços em ver o impacto da bimodalidade de nossas erupções:

In [ ]:
sns.histplot(data = tb_geyser, x = 'duration').set(title='Distribuição de Durações das Erupções')
plt.axvline(tb_geyser['duration'].median(), color = 'red');

Um pouco melhor que a média (pelo menos está dentro da segunda distribuição...) mas ainda assim não representa bem a diferença entre os dois tipos de erupção. A tratativa é a mesma que demos à média:

In [ ]:
sns.histplot(data = tb_geyser, x = 'duration', hue = 'kind').set(title='Distribuição de Durações das Erupções')
mask_short = tb_geyser['kind'] == 'short'
plt.axvline(tb_geyser.loc[mask_short,'duration'].median(), color = 'orange');
plt.axvline(tb_geyser.loc[~mask_short,'duration'].median(), color = 'blue');

Vamos comparar a média e a mediana neste mesmo gráfico:

In [ ]:
sns.histplot(data = tb_geyser, x = 'duration', hue = 'kind').set(title='Distribuição de Durações das Erupções')
mask_short = tb_geyser['kind'] == 'short'
plt.axvline(tb_geyser.loc[mask_short,'duration'].mean(), color = 'red');
plt.axvline(tb_geyser.loc[~mask_short,'duration'].mean(), color = 'purple');
plt.axvline(tb_geyser.loc[mask_short,'duration'].median(), color = 'orange');
plt.axvline(tb_geyser.loc[~mask_short,'duration'].median(), color = 'blue');

In [ ]:
tb_geyser.groupby('kind').agg(['mean', 'median'])

Embora a média e a mediana da distribuição das erupções longas sejam semelhantes, temos uma diferença maior entre as duas medidas para erupções curtas. Será que temos outliers entre essas erupções curtas? Veremos como podemos determinar isto ao final da aula!

#### Outros Quantis

A mediana é definida como a observação de uma variável tal que 50% das observações estejam abaixo dela e 50% acima. E se trocassemos essas proporções? Por exemplo, e se procurasse-mos a observação tal que 20% das observações estejam abaixo dela e 80% acima?

Essa medida chama-se quantil: o quantil 50 é a mediana, enquanto o quantil no exemplo acima é o quantil 20. Logo, o quantil X de uma variável é o valor da observação tal que X% das observações estejam abaixo deste valor!

Nem todos os quantis nascem iguais: três deles são mais *importantes* que o resto e são chamados de **quartis**. Os **quartis** separam nossas observações em 4 grupos de tamanho igual (com o mesmo número de observações):

1. O **primeiro quartil**, ou quantil 25, é o valor tal que 25% das observações estão abaixo deste valor;
1. O **segundo quartil**, ou mediana, é o valor tal que 50% das observações estão abaixo deste valor;
1. O **terceiro quartil**, ou mediana, é o valor tal que 75% das observações estão abaixo deste valor;

Podemos utilizar o método `.describe()` para ver os quartis de uma variável:

In [ ]:
tb_geyser.describe()

Também podemos utilizar a função `quantile()` para calcular qualquer quantil de uma variável:

In [ ]:
np.quantile(tb_geyser['waiting'], 0.25)

Podemos passar uma lista a esta função para extrair múltiplos quantis simultaneamente:

In [ ]:
np.quantile(tb_geyser['waiting'], [0.25, 0.5, 0.75])

ou podemos utilizar o método `.quantile()` dos `DataFrames`:

In [ ]:
tb_geyser['waiting'].quantile([0.25, 0.5, 0.75])

### Moda

As duas medidas que analisamos anteriormente só são definidas para variáveis continuas. E se precisarmos estimar a localização de uma variável categórica? A forma mais comum de fazê-lo é através da **moda**. A **moda** é definida como **o valor mais comum entre todas as observações**.

In [ ]:
tb_geyser['kind'].value_counts()

A **moda** de nossa variável `kind` é *long* com 172 observações. Para variáveis continuas não temos como calcular a moda diretamente: pequenas diferenças decimais tornam praticamente impossível que duas observações tenham exatamente o mesmo valor.

Para calcular a moda de variáveis continuas devemos **discretiza-las** antes: vamos utilizar a função `cut()` para discretizar as variáveis `waiting` e `duration`:

In [ ]:
tb_geyser['duration_cat'] = pd.cut(tb_geyser['duration'], 5)
tb_geyser['waiting_cat'] = pd.cut(tb_geyser['waiting'], 5)

Agora podemos calcular a moda de nossas variáveis numéricas:

In [ ]:
tb_geyser.select_dtypes(include = ['object', 'category']).mode()

No exemplo acima, podemos ver como a moda foi capaz de detectar a bi-modalidade da variável `duration`. Infelizmente, o uso da moda é muito dependente do número de categorias que utilizamos. Para contornar essa limitação podemos utilizar o **Estimador de Freedman Diaconis**, através da biblioteca NumPy:

In [ ]:
tb_geyser['duration_cat'] = pd.cut(tb_geyser['duration'], np.histogram_bin_edges(tb_geyser['duration'], bins = 'fd'))
tb_geyser['waiting_cat'] = pd.cut(tb_geyser['waiting'], np.histogram_bin_edges(tb_geyser['waiting'], bins = 'fd'))
tb_geyser.select_dtypes(include = ['object', 'category']).mode()

Agora vamos extrair utilizar o valor médio do intervalo da categoria como valor da nossa moda:

In [ ]:
moda_duration = tb_geyser.select_dtypes(include = ['object', 'category']).mode()['duration_cat'][0].mid
print(moda_duration)

Como podemos ver, mudando o número de faixas não temos mais a capacidade de detectar a bi-modalidade da distribuição. Independemente, a moda é muito útil em casos de distribuições multi-modais pois, no pior caso, representará o centro de uma das distribuições presente em nosso dado:

In [ ]:
moda_duration = tb_geyser.select_dtypes(include = ['object', 'category']).mode()['duration_cat'][0].mid
media_duration = tb_geyser['duration'].mean()
mediana_duration = tb_geyser['duration'].median()

In [ ]:

sns.histplot(data = tb_geyser, x = 'duration').set(title='Distribuição de Durações das Erupções')
mask_short = tb_geyser['kind'] == 'short'
plt.axvline(media_duration, color = 'red');
plt.axvline(mediana_duration, color = 'blue');
plt.axvline(moda_duration, color = 'black');

## Dispersão

A segunda categoria de estatísticas descritivas que veremos hoje são as **medidas de dispersão**. Equanto as **medidas de localização** representam **o valor ao redor do qual nossa variável está distribuída**, as **medidas de dispersão** representam **quanto as observações diferem da centralidade**. 

Visualmente podemos dizer que a localização é onde nosso histograma está localizado (em torno de 4, de 40, de 150...) e a dispersão é o quão *aberto* é o histograma (de 1 à 5, de 0 à 20, de -15 à 30).

### Desvio padrão

A primeira medida de dispersão é diretamente relacionada à média: o desvio padrão. Vamos ver como podemos calcular o desvio padrão para entender melhor o que está quantidade mede. Primeiro, vamos criar uma nova variável, chamada de `duration_desvio` que conterá a diferênça de cada observação para a média:

In [ ]:
tb_geyser_long = tb_geyser[tb_geyser['kind'] == 'long'].copy()

In [ ]:
tb_geyser_long['duration_desvio'] = tb_geyser_long['duration'] - np.mean(tb_geyser_long['duration'])

In [ ]:
tb_geyser_long['duration_desvio']

Essa nova variável representa o quão distante da média cada observação está. Podemos tentar resumir a dispersão calculando a média dessa nova variável!

In [ ]:
tb_geyser_long['duration_desvio'].mean()

Segundo essa lógica, o desvio é praticamente 0! O que aconteceu e como podemos corrigir isto?

In [ ]:
tb_geyser_long['duration_desvio_quadrado'] = tb_geyser_long['duration_desvio'] ** 2

In [ ]:
tb_geyser_long['duration_desvio_quadrado']

Agora, podemos calcular a média desse desvio ao quadrado:

In [ ]:
tb_geyser_long['duration_desvio_quadrado'].mean()

Infelizmente, nesse caso a unidade desta média não é mais minutos, e sim minutos ao quadrado! Vamos tomar a raiz quadrada para retornar a unidade para minutos:

In [ ]:
np.sqrt(tb_geyser_long['duration_desvio_quadrado'].mean())

Este número que calculamos é justamente o **desvio padrão** da variável `duration`! Podemos confirmar isso utilizando a função `std()`:

In [ ]:
np.std(tb_geyser_long['duration'])

ou o método `.std()` dos `DataFrames`

In [ ]:
tb_geyser_long['duration'].std()

A fórmula do desvio padrão que descreve as etapas que percorremos até aqui é bem simples:

$$\mu=\frac{\sum_{i=1}^n x_{i}}{n} $$

$$\sigma = \sqrt{\frac{\sum_{i=1}^n (x_{i} - \mu)^2}{n}}$$

A letra grega utilizada tradicionalmente para representar o desvio padrão é o *sigma*.

#### Aplicação I - Encontrando Outliers

Uma utilização comum do desvio padrão é a marcação de outliers. Podemos utilizar a distância de cada observação para a média, medida em desvios padrões da variável, como um indicador de quão *longe* do centro da distribuição uma observação está

In [ ]:
tb_geyser_long['score_outlier_dur'] = np.abs(tb_geyser_long['duration_desvio']/tb_geyser_long['duration'].std())
fig, ax = plt.subplots()
sns.histplot(data = tb_geyser_long, x = 'score_outlier_dur')
fig.suptitle('Distribuição de Distância da Média')
ax.set_xlabel('Desvios Padrões');

Uma regra comum utilizada marca todos os pontos a mais de 2 desvios padrões da média como outliers. Vamos realizar essa marcação para ver onde nossos outliers estão localizados:

In [ ]:
tb_geyser_long['outlier'] = np.where(tb_geyser_long['score_outlier_dur'] > 2, True, False)
fig, ax = plt.subplots()
sns.histplot(data = tb_geyser_long, x = 'duration', hue = 'outlier')
fig.suptitle('Distribuição de Duração da Erupção\n com Marcação de Outliers')
ax.set_xlabel('Minutos');

Podemos supor que as observações acima foram **mal classificadas**: talvez devessem pertencer ao grupo de erupções curtas!

É claro que essa marcação depende da *regra* que utiliza 2 desvios padrões como ponto de marcação dos outliers. Está regra é calcada na **Desigualdade de Chebyshev**, que garante, para qualquer distribuição (não-exótica), que 75% da população esta à dois desvios padrões da média. Esta desigualdade é extremamente conservadora: em uma distribuição que se aproxima de uma normal (quase tudo, pelo teorema do limite central), essa porcentagem é mais próxima de 95%!

#### Aplicação II - Coeficiente de Variação

Embora o desvio padrão seja mensurado na mesma unidade da variável para qual foi calculado (minutos no exemplo acima), em aplicações reais ele é muitas vezes *condicionado* à média. Por exemplo, em um mercado queremos saber qual produto tem mais variabilidade nas vendas - muitas vezes vamos observar que o produto que, em média, vende mais tem um desvio padrão maior. **Isso representa uma dispersão maior?**

Não necessariamente! Imagine que o **produto A** vende 100 unidades em média, com um desvio padrão = 5, enquanto o **produto B** vende em média 10 unidades com um desvio padrão = 4. **Qual produto, em sua opinião, apresenta mais variabilidade?**

Uma forma de poder comparar a dispersão de dois grupos com médias distintas é através da **proporção entre o desvio padrão e a média** - esta medida se chama **coeficiente de variação**!

In [ ]:
tb_geyser.groupby('kind')['duration'].mean()

In [ ]:
tb_geyser.groupby('kind')['duration'].std()

In [ ]:
tb_geyser_ed = tb_geyser.groupby('kind')['duration'].agg(['mean', 'std']).reset_index()
tb_geyser_ed

In [ ]:
tb_geyser_ed['coef_var'] = tb_geyser_ed['std']/tb_geyser_ed['mean']
tb_geyser_ed

### Limitações do Desvio Padrão

A principal limitação do desvio padrão surge quando consideramos distribuições assimétricas: como utilizamos o desvio padrão a partir da média, para distribuições simétricas estamos estimando uma dispersão simétrica.

Vamos visualizar este problema nas erupções de curta duração:

In [ ]:
tb_geyser_short = tb_geyser[tb_geyser['kind'] == 'short'].copy()
fig, ax = plt.subplots()
sns.histplot(data = tb_geyser_short, x = 'duration')
plt.axvline(tb_geyser_short['duration'].mean(), color = 'red')
plt.axvline(tb_geyser_short['duration'].mean() - 2 * tb_geyser_short['duration'].std(), color = 'black')
plt.axvline(tb_geyser_short['duration'].mean() + 2 * tb_geyser_short['duration'].std(), color = 'black')
fig.suptitle('Distribuição da Duração de Erupções\n com região à 2 desvios padrões da média');

Como podemos ver no gráfico acima, a faixa projetada para a duração das erupções está deslocada em relação à distribuição observada: como a distribuição é assimétrica, ou seja, tem mais observações acima da média do que abaixo, a simetria da média e do desvio padrão não capturam a assimetria da dispersão.

## IQR (Amplitude interquartil)

Enquanto o desvio padrão é uma medida de dispersão relacionada à média, o **IQR** (*inter-quartile range*) é uma medida de dispersão relacionada à mediana. O IQR é definido como a diferença entre o primeiro e o terceiro quartil, ou seja, é a distância que contém 50% das observações de nosso dataset:

In [ ]:
dur_p25 = np.quantile(tb_geyser_short['duration'], 0.25)
print(dur_p25)

In [ ]:
dur_p75 = np.quantile(tb_geyser_short['duration'], 0.75)
print(dur_p75)

In [ ]:
print(dur_p75 - dur_p25)

Vamos construir uma função para calcular, a partir de um vetor de observações numéricas, a distância inter-quartil:

In [ ]:
def iqr(vetor):
    dur_p25 = np.quantile(vetor, 0.25)
    dur_p75 = np.quantile(vetor, 0.75)
    return dur_p75 - dur_p25

In [ ]:
iqr(tb_geyser_short['duration'])

#### Aplicação III - Econtrando Outliers II

Como a combinação média/desvio padrão apresentam dificuldade ao lidar com dados de distribuições assimétricas, vamos utilizar o IQR e os quartis para procurar outliers na duração de erupções do tipo curto.

Vamos utilizar a distância inter-quartil para calcular limites inferiores e superiores em relação ao primeiro e terceiro quartis:

In [ ]:
iqr_short = iqr(tb_geyser_short['duration'])
lim_inf = tb_geyser_short['duration'].quantile(0.25) - 1.5*iqr_short
lim_sup  = tb_geyser_short['duration'].quantile(0.75) + 1.5*iqr_short

No código acima podemos ver que o limite inferior é:
$$Q25 - 1.5*IQR$$
enquanto o limite superior é:
$$Q75 + 1.5*IQR$$
Podemos ver a assimetria dessa medida comparando a distância do limite inferior e superior para a mediana:

In [ ]:
tb_geyser_short['duration'].median() - lim_inf

In [ ]:
lim_sup - tb_geyser_short['duration'].median()

Agora vamos utiliza-los para classificar as observações em outliers:

In [ ]:
obs_lim_inf = tb_geyser_short['duration'] < lim_inf
obs_lim_sup = tb_geyser_short['duration'] > lim_sup
tb_geyser_short['outlier'] = False
tb_geyser_short.loc[obs_lim_inf, 'outlier'] = True
tb_geyser_short.loc[obs_lim_sup, 'outlier'] = True

fig, ax = plt.subplots()
sns.histplot(data = tb_geyser_short, x = 'duration', hue = 'outlier')
fig.suptitle('Distribuição de Duração da Erupção\n com Marcação de Outliers')
ax.set_xlabel('Minutos');

Vamos unir as duas classificações de outlier para analisarmos o padrão geral de outliers em duração de erupções:

In [ ]:
tb_geyser_out = pd.concat([tb_geyser_short, tb_geyser_long], axis = 0)

fig, ax = plt.subplots()
sns.histplot(data = tb_geyser_out, x = 'duration', hue = 'outlier')
fig.suptitle('Distribuição de Duração da Erupção\n com Marcação de Outliers')
ax.set_xlabel('Minutos');

Ao analisar o padrão global podemos notar que existe uma região de duração das erupções onde não é claro se elas deveriam ser classificadas como longas ou curtas!

## Apendice - Usando Dados Sintéticos

Uma outra forma de visualizar diferenças entre distribuições com médias e desvio padrões diferentes é utilizando a simulação de Monte Carlo. Veremos mais sobre este assunto na próxima aula, quando aprenderemos como podemos gerar números aleatórios a partir de uma distribuição probabílistica conhecida.

Por enquanto, vocês podem acompanhar após a aula as simulação de 3 distribuições: 

1. A primeira com média 0 e desvio padrão 1;
1. A segunda com média 0 e desvio padrão 2;
1. A terceira com média 2 e desvio padrão 1.

Com essas simulações podemos visualizar diretamente os efeitos das diferenças de localização (variação da média) e de dispersão (variação do desvio padrão).

### Simulando dados

Primeiro, vamos utilizar o sub-módulo `.random` da biblioteca `numpy` para criar amostras de distribuições normais com os parâmetros descritos acima.

In [ ]:
mu_0_sd_1 = np.random.normal(0, 1, 1000)
mu_0_sd_2 = np.random.normal(0, 2, 1000)
mu_2_sd_1 = np.random.normal(2, 1, 1000)

### Diferença de localização

Percebam o que acontece quando comparamos duas distribuições com médias diferentes (0 e 2) e desvios padrões iguais (1):

In [ ]:
plt.figure(figsize=(8,6));
plt.hist(mu_2_sd_1, bins=10, alpha=0.5, label="mu_2_sd_1");
plt.hist(mu_0_sd_1, bins=10, alpha=0.5, label="mu_0_sd_1");
plt.axvline(np.mean(mu_0_sd_1), color = 'blue');
plt.axvline(np.mean(mu_2_sd_1), color = 'red');
plt.xlabel("Dados Simulados", size=14)
plt.ylabel("Contagem", size=14);
plt.title("Distribuições Normais mu = 0 e mu = 2, sigma = 1");
plt.legend(loc='upper right');

### Diferença de dispersão

Percebam o que acontece quando comparamos duas distribuições com desvio padrões diferentes (1 e 2) e médias iguais (0):

In [ ]:
plt.figure(figsize=(8,6));
plt.hist(mu_0_sd_2, bins=10, alpha=0.5, label="mu_0_sd_2");
plt.hist(mu_0_sd_1, bins=10, alpha=0.5, label="mu_0_sd_1");
plt.axvline(np.mean(mu_0_sd_1), color = 'blue');
plt.axvline(np.mean(mu_0_sd_2), color = 'red');
plt.xlabel("Dados Simulados", size=14)
plt.ylabel("Contagem", size=14);
plt.title("Distribuições Normais mu = 0, sigma = 1, sigma = 2");
plt.legend(loc='upper right');

### Diferença de Dispersão e Localização

Percebam o que acontece quando comparamos duas distribuições com desvio padrões diferentes (1 e 2) e médias diferentes (2, 0):

In [ ]:
plt.figure(figsize=(8,6));
plt.hist(mu_0_sd_2, bins=10, alpha=0.5, label="mu_0_sd_2");
plt.hist(mu_2_sd_1, bins=10, alpha=0.5, label="mu_0_sd_1");
plt.axvline(np.mean(mu_2_sd_1), color = 'blue');
plt.axvline(np.mean(mu_0_sd_2), color = 'red');
plt.xlabel("Dados Simulados", size=14)
plt.ylabel("Contagem", size=14);
plt.title("Distribuições Normais mu = 0, sigma = 1, sigma = 2");
plt.legend(loc='upper right');

### Vizualizando Indicadores Robustos

Além de utilizarmos histogramas, podemos utilizar boxplots para visualizar os quartis, o IQR e os outliers (segundo a metodologia utilizada na Aplicação III):

1. A *caixa* no meio de um boxplot é o IQR, seu limite inferior é o Q25 e seu limite superior o Q75.
1. Os *bigodes* que saem da caixa representam os limites inferiores e superiores (Q25 - 1.5 * IQR e Q75 + 1.5 * IQR)
1. Os asteriscos fora dos limites dos bigodes são outliers.

In [ ]:
plt.figure(figsize=(8,6));
plt.boxplot([mu_0_sd_1, mu_2_sd_1, mu_0_sd_2],
           labels = ['mu_0_sd_1', 'mu_2_sd_1', 'mu_0_sd_2']);
plt.ylabel('Dado Simulado');
plt.xlabel('Distribuição');
plt.suptitle('Boxplots de Distribuições');